Auteur: Michaël Leroy


Modification du traitement pour la création des edges graph communes, essai sur geodataframe pour profiter de la vectorisation:
        testé jusqu'a la recherche de paths... qui n'est pas vect et du coup hyper long (j'ai killé au bout de 3h)

    Création geodataframe Edges:

        calcul des edges attributes:
            - shortest path depuis le graph routier => distance, durée
            - geométrie: somme des LineString
            - length : somme des longeurs de chaque tronçons en mètres

[+] A faire:
        
        - prolonger les geometries des linestrings routes jusqu'au points centraux communes si on veut se servir des tracés sur une carte, sinon pas grave.

[-] Cons:
        
        40 min pour 250 nodes et 1219 edges

        -> temps estimé pour 170188 edges ~76h


In [1]:
class CFG:
    area = [
        'Auvergne-Rhône-Alpes',
    'Bourgogne-Franche-Comté',
    'Brittany',
    'Centre-Val de Loire',
    'Grand Est',
    'Hauts-de-France',
    'Île-de-France',
    'Normandie',
    'Nouvelle-Aquitaine',
    'Occitanie',
    'Pays de la Loire',
    'Provence-Alpes-Côte d\'Azur'
]
    area_name = 'france'      #'_'.join(area)

    

    bornes = False

    viz = True

    roads = (
        '|motorway'
        '|motorway_link'
        '|trunk'
        '|trunk_link'
        '|primary'
        '|primary_link'
        '|secondary'
        '|secondary_link'
        '|tertiary'
         '|tertiary_link'
    )

        # '|primary'
        # '|motorway_link'
        
        # '|primary_link'

        # '|secondary'
        # '|secondary_link'
        
        
        #  '|tertiary'
        #  '|tertiary_link'

    data_path = './osm_datas/'

import joblib
joblib.dump(CFG.area, 'CFG.pkl')    

import os
os.environ['USE_PYGEOS'] = '0'

%load_ext autoreload
%autoreload


%aimport OSM_helper_functions
%aimport OCM_helper_functions

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import osmnx as ox
import networkx as nx
from tqdm.notebook import tqdm
from OSM_helper_functions import *
from OCM_helper_functions import *


ox.settings.log_console=True
ox.settings.use_cache=True
''' To retrieve http data from OSMnx and cache them locally
    before building graph. Raise and exception if requests
    are all done. intercept then set to False and retry.'''
# ox.settings.cache_only_mode=True 

Administratives = {
    'Auvergne-Rhône-Alpes': ['Ain', 'Allier', 'Ardèche', 'Cantal', 'Drôme', 'Isère', 'Loire', 'Haute-Loire', 'Puy-de-Dôme', 'Rhône', 'Savoie', 'Haute-Savoie'],
    'Bourgogne-Franche-Comté': ['Côte-d\'Or', 'Doubs', 'Jura', 'Nièvre', 'Haute-Saône', 'Saône-et-Loire', 'Yonne', 'Territoire de Belfort'],
    'Brittany': ['Côtes-d\'Armor', 'Finistère', 'Ille-et-Vilaine', 'Morbihan'],
    'Centre-Val de Loire': ['Cher', 'Eure-et-Loir', 'Indre', 'Indre-et-Loire', 'Loir-et-Cher', 'Loiret'],
    # 'Corse': ['Corse-du-Sud', 'Haute-Corse'],
    'Grand Est': ['Ardennes', 'Aube', 'Marne', 'Haute-Marne', 'Meurthe-et-Moselle', 'Meuse', 'Moselle', 'Bas-Rhin', 'Haut-Rhin', 'Vosges'],
    'Hauts-de-France': ['Aisne', 'Nord', 'Oise', 'Pas-de-Calais', 'Somme'],
    'Île-de-France': ['Paris', 'Seine-et-Marne', 'Yvelines', 'Essonne', 'Hauts-de-Seine', 'Seine-Saint-Denis', 'Val-de-Marne', 'Val-d\'Oise'],
    'Normandie': ['Calvados', 'Eure', 'Manche', 'Orne', 'Seine-Maritime'],
    'Nouvelle-Aquitaine': ['Charente', 'Charente-Maritime', 'Corrèze', 'Creuse', 'Dordogne', 'Gironde', 'Landes', 'Lot-et-Garonne', 'Pyrénées-Atlantiques', 'Deux-Sèvres', 'Vienne', 'Haute-Vienne'],
    'Occitanie': ['Ariège', 'Aude', 'Aveyron', 'Gard', 'Haute-Garonne', 'Gers', 'Hérault', 'Lot', 'Lozère', 'Hautes-Pyrénées', 'Pyrénées-Orientales', 'Tarn', 'Tarn-et-Garonne'],
    'Pays de la Loire': ['Loire-Atlantique', 'Maine-et-Loire', 'Mayenne', 'Sarthe', 'Vendée'],
    'Provence-Alpes-Côte d\'Azur': ['Alpes-de-Haute-Provence', 'Hautes-Alpes', 'Alpes-Maritimes', 'Bouches-du-Rhône', 'Var', 'Vaucluse']
}

Regions = [
    'Auvergne-Rhône-Alpes',
    'Bourgogne-Franche-Comté',
    'Brittany',
    'Centre-Val de Loire',
    # 'Corse',
    'Grand Est',
    'Hauts-de-France',
    'Île-de-France',
    'Normandie',
    'Nouvelle-Aquitaine',
    'Occitanie',
    'Pays de la Loire',
    'Provence-Alpes-Côte d\'Azur'
]

Name2num = {
    'Ain': '01',
    'Aisne': '02',
    'Allier': '03',
    'Alpes-de-Haute-Provence': '04',
    'Hautes-Alpes': '05',
    'Alpes-Maritimes': '06',
    'Ardèche': '07',
    'Ardennes': '08',
    'Ariège': '09',
    'Aube': '10',
    'Aude': '11',
    'Aveyron': '12',
    'Bouches-du-Rhône': '13',
    'Calvados': '14',
    'Cantal': '15',
    'Charente': '16',
    'Charente-Maritime': '17',
    'Cher': '18',
    'Corrèze': '19',
    # 'Corse-du-Sud': '2A',
    # 'Haute-Corse': '2B',
    'Côte-d\'Or': '21',
    'Côtes-d\'Armor': '22',
    'Creuse': '23',
    'Dordogne': '24',
    'Doubs': '25',
    'Drôme': '26',
    'Eure': '27',
    'Eure-et-Loir': '28',
    'Finistère': '29',
    'Gard': '30',
    'Haute-Garonne': '31',
    'Gers': '32',
    'Gironde': '33',
    'Hérault': '34',
    'Ille-et-Vilaine': '35',
    'Indre': '36',
    'Indre-et-Loire': '37',
    'Isère': '38',
    'Jura': '39',
    'Landes': '40',
    'Loir-et-Cher': '41',
    'Loire': '42',
    'Haute-Loire': '43',
    'Loire-Atlantique': '44',
    'Loiret': '45',
    'Lot': '46',
    'Lot-et-Garonne': '47',
    'Lozère': '48',
    'Maine-et-Loire': '49',
    'Manche': '50',
    'Marne': '51',
    'Haute-Marne': '52',
    'Mayenne': '53',
    'Meurthe-et-Moselle': '54',
    'Meuse': '55',
    'Morbihan': '56',
    'Moselle': '57',
    'Nièvre': '58',
    'Nord': '59',
    'Oise': '60',
    'Orne': '61',
    'Pas-de-Calais': '62',
    'Puy-de-Dôme': '63',
    'Pyrénées-Atlantiques': '64',
    'Hautes-Pyrénées': '65',
    'Pyrénées-Orientales': '66',
    'Bas-Rhin': '67',
    'Haut-Rhin': '68',
    'Rhône': '69',
    'Haute-Saône': '70',
    'Saône-et-Loire': '71',
    'Sarthe': '72',
    'Savoie': '73',
    'Haute-Savoie': '74',
    'Paris': '75',
    'Seine-Maritime': '76',
    'Seine-et-Marne': '77',
    'Yvelines': '78',
    'Deux-Sèvres': '79',
    'Somme': '80',
    'Tarn': '81',
    'Tarn-et-Garonne': '82',
    'Var': '83',
    'Vaucluse': '84',
    'Vendée': '85',
    'Vienne': '86',
    'Haute-Vienne': '87',
    'Vosges': '88',
    'Yonne': '89',
    'Territoire de Belfort': '90',
    'Essonne': '91',
    'Hauts-de-Seine': '92',
    'Seine-Saint-Denis': '93',
    'Val-de-Marne': '94',
    'Val-d\'Oise': '95'
}


AREA = []
for area in CFG.area:
    AREA.extend(Administratives[area] )
# AREA

C:\Users\demo\AppData\Roaming\Python\Python39\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:

# # Nodes = joblib.load(os.path.join(CFG.data_path, f'[{CFG.area_name}]Gnodes_communes.pkl'))
# Edges = gpd.read_feather(os.path.join(CFG.data_path, f'[{CFG.area_name}]Gedges_communes_0.feather'))
# # G_communes = ox.graph_from_gdfs(Nodes, Edges)

# Edges.reset_index(inplace=True)

# Edges.sample(2)

In [3]:
# Edges.info()

In [4]:
# Nodes_routier = joblib.load( os.path.join(CFG.data_path, f'[{CFG.area_name}]Gnodes.pkl'))
Graph_routier = ox.graph_from_gdfs(
    joblib.load( os.path.join(CFG.data_path, f'[{CFG.area_name}]Gnodes.pkl')), 
    joblib.load( os.path.join(CFG.data_path, f'[{CFG.area_name}]Gedges.pkl'))
    )

In [5]:
# Edges = Edges.sample(1000)

In [6]:
# # TODO: 1 voir pour vectoriser cette étape
# #            - sub classer la classe source 
# #            _ modiifer le code source

# from tqdm.notebook import tqdm
# tqdm.pandas()


def get_shortest_path( arg: tuple[int, int]) -> list: #, graph_routier=Graph_routier):
    return nx.shortest_path(Graph_routier, 
                            source=arg[-2], 
                            target=arg[-1], 
                            weight='travel_time', 
                            method='dijkstra')

# display(get_path((1424062767, 1651507178)))    


# %time Edges['path'] = Edges.src_trgt.progress_apply(get_shortest_path)
# Edges.sample(2)

In [7]:
# from tqdm.notebook import tqdm
# tqdm.pandas()
# from math import sqrt, pow


def dist(a:int, b: int) -> float:
    node_a = Graph_routier.nodes[a]
    node_b = Graph_routier.nodes[b]

    (x1, y1) = node_a['x'], node_a['y']
    (x2, y2) = node_b['x'], node_b['y']
    return sqrt(pow(x1 - x2, 2) + pow(y1 - y2, 2) )



def get_astar_path( arg: tuple[int, int]) -> list: #, graph_routier=Graph_routier):
    return nx.astar_path(Graph_routier, 
                            source=arg[-2], 
                            target=arg[-1], 
                            weight='travel_time', 
                            heuristic=dist)

# display(get_path((1424062767, 1651507178)))    


# %time Edges['path'] = Edges.src_trgt.progress_apply(get_astar_path)
# Edges.sample(2)

------------------------------

In [8]:
from tqdm.notebook import tqdm
tqdm.pandas(position=1, leave=False)


In [9]:
batch_size = 10000

# , initial=Edges.path.notna().sum()
with tqdm(#total=135685, 
          initial=(gpd
                   .read_feather(
                       os.path.join(
                           CFG.data_path, 
                           f'[{CFG.area_name}]Gedges_communes_0_v(ox).feather'
                        )
                    )
                    .path.notna().sum()
          ), 
          desc = 'edges', 
          position=0,
          colour ='green') as t:
    while True:
        # Load Edges
        Edges = gpd.read_feather(os.path.join(CFG.data_path, f'[{CFG.area_name}]Gedges_communes_0_v(ox).feather'))
        Edges.reset_index(inplace=True)

        # Split Edges to done and to_do
        Edges_done = Edges[~Edges.path.isna()].copy()
        # t.reset(total=135685)
        # t.update(Edges_done.shape[0])
        t.refresh()
        t.update(batch_size)
        Edges_to_do = Edges[Edges.path.isna()].copy()
        if Edges_done.shape[0] == Edges.shape[0]:
            break
        # else:
        #     print(f'Done {Edges_done.shape[0]/Edges.shape[0]:.5%}',end='\r')

        del Edges

        # Take a batch of edges
        Edges_working = Edges_to_do.iloc[:batch_size]
        Edges_to_do  =  Edges_to_do.iloc[batch_size:]
        # display(Edges_done.index, Edges_working.index, Edges_to_do.index)
        
        # Compute paths
        try:
            Edges_working['path'] = Edges_working.src_trgt.progress_apply(get_shortest_path)
        except:
            print(f'Path not found | batch {Edges_working.index[0]} to {Edges_working.index[-1]}')
            break

        # Save progress
        (
        pd.concat([Edges_done, Edges_working, Edges_to_do], axis=0)
        .set_index(['u','v'])
        .to_feather(
            os.path.join(
                CFG.data_path, 
                f'[{CFG.area_name}]Gedges_communes_0_v(ox).feather'
            )
        )
        )
        del Edges_done, Edges_to_do, Edges_working

edges: 40000it [00:00, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

...

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/5450 [00:00<?, ?it/s]

In [10]:
Edges = gpd.read_feather(os.path.join(CFG.data_path, f'[{CFG.area_name}]Gedges_communes_0_v(ox).feather'))
Edges.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
MultiIndex: 205450 entries, (2531584, 2176035) to (2741108, 2740963)
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype   
---  ------    --------------   -----   
 0   key       205450 non-null  int64   
 1   geometry  205450 non-null  geometry
 2   src_trgt  205450 non-null  object  
 3   path      205450 non-null  object  
dtypes: geometry(1), int64(1), object(2)
memory usage: 10.4+ MB


In [11]:
Edges

key                                           geometry  \
u       v                                                                 
2531584 2176035    0  LINESTRING (-0.81977 43.46144, -0.84879 43.43624)   
        167704     0  LINESTRING (-0.81977 43.46144, -0.77384 43.48750)   
        1380951    0  LINESTRING (-0.81977 43.46144, -0.79568 43.44181)   
        2531635    0  LINESTRING (-0.81977 43.46144, -0.83523 43.49858)   
279376  279375     0    LINESTRING (0.77749 47.73042, 0.73910 47.74537)   
...              ...                                                ...   
2741108 5811913    0    LINESTRING (3.50709 48.86596, 3.48873 48.89115)   
        2742275    0    LINESTRING (3.50709 48.86596, 3.60070 48.80919)   
        1769723    0    LINESTRING (3.50709 48.86596, 3.53820 48.87088)   
        2742573    0    LINESTRING (3.50709 48.86596, 3.53126 48.79645)   
        2740963    0    LINESTRING (3.50709 48.86596, 3.49967 48.85079)   

                                 src_trgt  \
u       v                                   
2531584 2176035    [859309184, 504799281]   
        167704    [859309184, 3957848575]   
        1380951   [859309184, 1976671808]   
        2531635   [859309184, 1975118728]   
279376  279375    [281892451, 1372889494]   
...                                   ...   
2741108 5811913  [3427753813, 3737538194]   
        2742275  [3427753813, 1296924098]   
        1769723  [3427753813, 5928930665]   
        2742573  [3427753813, 1092502737]   
        2740963  [3427753813, 8335004268]   

                                                              path  
u       v                                                           
2531584 2176035      [859309184, 1976671808, 504799280, 504799281]  
        167704   [859309184, 1976671808, 8002586203, 859309092,...  
        1380951                            [859309184, 1976671808]  
        2531635  [859309184, 1976671808, 8002586203, 859309092,...  
279376  279375                  [281892451, 281890718, 1372889494]  
...                                                            ...  
2741108 5811913   [3427753813, 2186352923, 1116200620, 3737538194]  
        2742275  [3427753813, 2186352923, 2186352928, 109247384...  
        1769723  [3427753813, 2186352923, 2186352928, 109247384...  
        2742573  [3427753813, 2186352923, 2186352928, 109247384...  
        2740963  [3427753813, 2186352923, 2186352928, 103871966...  

[205450 rows x 4 columns]

In [12]:
Edges = Edges.reset_index()
# Edges.u = Edges.u.astype(int)
# Edges.v = Edges.v.astype(int)

In [13]:
(
Edges
.set_index(['u','v','key'])
.to_feather(
            os.path.join(
                CFG.data_path, 
                f'[{CFG.area_name}]Gedges_communes_C21.feather'
            )
        )
)

In [14]:
Edges = gpd.read_feather(os.path.join(CFG.data_path, f'[{CFG.area_name}]Gedges_communes_C21.feather'))
Edges.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
MultiIndex: 205450 entries, (2531584, 2176035, 0) to (2741108, 2740963, 0)
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype   
---  ------    --------------   -----   
 0   geometry  205450 non-null  geometry
 1   src_trgt  205450 non-null  object  
 2   path      205450 non-null  object  
dtypes: geometry(1), object(2)
memory usage: 9.0+ MB


----------------------------------------

In [15]:
**********

SyntaxError: invalid syntax (1655670802.py, line 1)

In [ ]:
# from tqdm.notebook import tqdm
# tqdm.pandas()


# # def get_path_length( path, nodes=Nodes, nodes_routier=Nodes_routier, graph_routier=Graph_routier ):
# def get_path_length( path):
#     length = sum(
#         ox.utils_graph.get_route_edge_attributes(
#                     Graph_routier, 
#                     path, 
#                     attribute='length', 
#                     minimize_key='length', 
#                     retrieve_default=None)
#     )
#     return length



# Edges['length'] = Edges.path.progress_apply(get_path_length)
# Edges.sample(2)

In [ ]:
# # def get_path_line( path, nodes=Nodes, nodes_routier=Nodes_routier, graph_routier=Graph_routier ):
# def get_path_line( path ):
#     lines = ox.utils_graph.get_route_edge_attributes(
#                     Graph_routier, 
#                     path, 
#                     attribute='geometry', 
#                     minimize_key='length', 
#                     retrieve_default=None)
#     multi_line = MultiLineString(lines)
#     merged_line = ops.linemerge(multi_line)
#     return merged_line



# Edges['geometry'] = Edges.path.progress_apply(get_path_line)
# Edges.sample(2)

In [ ]:
# # # extend edges geometry to nodes center point
# # new_point = Point(-1, -1)
# # extended_line = LineString([new_point.coords[0], *line.coords])

# Nodes.head(1)

In [ ]:
# TODO:  supprimer les colonnes intermediaires avant la sauvegarde
#        sauvegarder les paths

In [ ]:
# joblib.dump(
#             Edges.path,
#             os.path.join(CFG.data_path, f'[{CFG.area_name}]Gedges_communes_paths.pkl')
#         )

In [ ]:
# joblib.dump(
#             Edges.drop('path', axis=1),
#             os.path.join(CFG.data_path, f'[{CFG.area_name}]Gedges_communes.pkl')
#         )

--------------------------------